# Save and retrieve jobs

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.3.1
qiskit-ibm-runtime~=0.34.0
qiskit-aer~=0.15.1
qiskit-serverless~=0.18.1
qiskit-ibm-catalog~=0.2
qiskit-addon-sqd~=0.8.1
qiskit-addon-utils~=0.1.0
qiskit-addon-mpf~=0.2.0
scipy~=1.14.1
qiskit-addon-aqc-tensor~=0.1.2
qiskit-addon-obp~=0.1.0
scipy~=1.14.1
pyscf~=2.7.0
```
</details>

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to file and retrieve results of past jobs from IBM Quantum&trade; so your next session can continue where you left off.

## Retrieve jobs from IBM Quantum

IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs through the [Workloads dashboard](https://quantum.ibm.com/workloads).

To find a job programatically, use the [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) method. By default, this returns the most recent jobs, but you can also filter jobs by backend name, creation date, and more. The following cell finds any jobs submitted in the last three months. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [1]:
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

service = QiskitRuntimeService()
jobs_in_last_three_months = service.jobs(created_after=three_months_ago)
jobs_in_last_three_months[:3]  # show first three jobs

[<RuntimeJob('cxg030c6t010008cme5g', 'estimator')>,
 <RuntimeJob('cxg02zvvw7kg008s3vk0', 'estimator')>,
 <RuntimeJob('cxg02z36t010008cme50', 'estimator')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) in the API documentation.

Once you have the job ID, use the [`QiskitRuntimeService.job`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#job) method to retrieve it.

In [2]:
# Get ID of most recent successful job for demonstration.
# This will not work if you've never successfully run a job.
successful_job = next(
    j for j in service.jobs(limit=1000) if j.status().name == "DONE"
)
job_id = successful_job.job_id()
print(job_id)

/tmp/ipykernel_5348/3090032923.py:4: DeprecationWarning: In a future release of qiskit-ibm-runtime no sooner than 3 months after the release date of 0.30.0, RuntimeJob.status() will be returned as a string instead of an instance of `JobStatus`. To prepare for this change, you can use the idiom `status.name if isinstance(status, JobStatus) else status`.
  j for j in service.jobs(limit=1000) if j.status().name == "DONE"


cxg030c6t010008cme5g


In [3]:
retrieved_job = service.job(job_id)
retrieved_job.result()

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(6,), dtype=float64>), stds=np.ndarray(<shape=(6,), dtype=float64>), evs_noise_factors=np.ndarray(<shape=(6, 2), dtype=float64>), stds_noise_factors=np.ndarray(<shape=(6, 2), dtype=float64>), ensemble_stds_noise_factors=np.ndarray(<shape=(6, 2), dtype=float64>), evs_extrapolated=np.ndarray(<shape=(6, 2, 3), dtype=float64>), stds_extrapolated=np.ndarray(<shape=(6, 2, 3), dtype=float64>), shape=(6,)), metadata={'shots': 100000, 'target_precision': 0.003162277660168379, 'circuit_metadata': {}, 'resilience': {'zne': {'extrapolator': array(['exponential', 'exponential', 'exponential', 'exponential',
       'exponential', 'exponential'], dtype='<U11')}, 'layer_noise': {'noise_overhead': 2.3541035574729365, 'total_mitigated_layers': 6, 'unique_mitigated_layers': 2, 'unique_mitigated_layers_noise_overhead': [1.1356441238948969, 1.1896748089636464]}}, 'num_randomizations': 1000})], metadata={'dynamical_decoupling': {'enable': False, 

<CodeAssistantAdmonition
  tagLine="Always forgetting how to retrieve a job? Try this prompt with Qiskit Code Assistant:"
  prompts={[
    "# Retrieve job JOB_ID from IBM Runtime and print the result"
  ]}
/>

## Save results to disk

You may also want to save results to disk. To do this, use Python's built-in JSON library with Qiskit Runtime's encoders.

In [4]:
import json
from qiskit_ibm_runtime import RuntimeEncoder

with open("result.json", "w") as file:
    json.dump(retrieved_job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [5]:
from qiskit_ibm_runtime import RuntimeDecoder

with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(6,), dtype=float64>), stds=np.ndarray(<shape=(6,), dtype=float64>), evs_noise_factors=np.ndarray(<shape=(6, 2), dtype=float64>), stds_noise_factors=np.ndarray(<shape=(6, 2), dtype=float64>), ensemble_stds_noise_factors=np.ndarray(<shape=(6, 2), dtype=float64>), evs_extrapolated=np.ndarray(<shape=(6, 2, 3), dtype=float64>), stds_extrapolated=np.ndarray(<shape=(6, 2, 3), dtype=float64>), shape=(6,)), metadata={'shots': 100000, 'target_precision': 0.003162277660168379, 'circuit_metadata': {}, 'resilience': {'zne': {'extrapolator': array(['exponential', 'exponential', 'exponential', 'exponential',
       'exponential', 'exponential'], dtype='<U11')}, 'layer_noise': {'noise_overhead': 2.3541035574729365, 'total_mitigated_layers': 6, 'unique_mitigated_layers': 2, 'unique_mitigated_layers_noise_overhead': [1.1356441238948969, 1.1896748089636464]}}, 'num_randomizations': 1000})], metadata={'dynamical_decoupling': {'enable': False, 